In [1]:
# import IPython;IPython.embed()
from __future__ import division

import os
import sys
import time
import datetime
import argparse

# Import things from actual YOLO folder
sys.path.insert(0, '/root/PyTorch-YOLOv3')

from models import *
from utils.utils import *
from utils.datasets import *


import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

print("Imported all successfully!")

Imported all successfully!


In [2]:
image_folder = '../PyTorch-YOLOv3/data/samples'
config_path = '../PyTorch-YOLOv3/config/yolov3.cfg'
weights_path = '../PyTorch-YOLOv3/weights/yolov3.weights'
class_path = '../PyTorch-YOLOv3/data/coco.names'
output_folder = "OUTPUT_IMAGES"
conf_thres = 0.8
nms_thres = 0.4
batch_size = 1
n_cpu = 8
img_size = 416
use_cuda = True

In [ ]:
cuda = torch.cuda.is_available() and use_cuda # Whether the gpu is available
os.makedirs(output_folder, exist_ok=True)   
model = Darknet(config_path, img_size=img_size) # Set up model
model.load_weights(weights_path)
if cuda:
    model.cuda()
model.eval() # Set in evaluation mode
dataloader = DataLoader(ImageFolder(image_folder, img_size=img_size), batch_size=batch_size, shuffle=False, num_workers=n_cpu)
classes = load_classes(class_path) # Extracts class labels from file
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
imgs = []           # Stores image paths
img_detections = [] # Stores detections for each image index
print ("\nPerforming object detection:")
prev_time = time.time()
for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
    # Configure input
    input_imgs = Variable(input_imgs.type(Tensor))
    
    import IPython;IPython.embed()
    
    # Get detections
    with torch.no_grad():
        detections = model(input_imgs)
        detections = non_max_suppression(detections, 80, conf_thres, nms_thres)
    # Log progress
    current_time = time.time()
    inference_time = datetime.timedelta(seconds=current_time - prev_time)
    prev_time = current_time
    print ('\t+ Batch %d, Inference Time: %s' % (batch_i, inference_time))
    # Save image and detections
    imgs.extend(img_paths)
    img_detections.extend(detections)
print("Done with inferrence")
#import IPython;IPython.embed()

/root/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))



Performing object detection:
Python 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
Type 'copyright', 'credits' or 'license' for more information
IPython 6.1.0 -- An enhanced Interactive Python. Type '?' for help.

In [1]: 

In [1]: type(input_imgs)
Out[1]: torch.Tensor

In [2]: input_imgs.type()
Out[2]: 'torch.FloatTensor'

In [3]: input_imgs.size
Out[3]: <function Tensor.size>

In [4]: input_imgs.shape
Out[4]: torch.Size([1, 3, 416, 416])



In [ ]:
# Bounding-box colors
cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

print ('\nSaving images:')
# Iterate through images and save plot of detections
for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):
    print ("(%d) Image: '%s'" % (img_i, path))

    # Create plot
    img = np.array(Image.open(path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # The amount of padding that was added
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    # Image height and width after padding is removed
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x

    # Draw bounding boxes and labels of detections
    if detections is not None:
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        bbox_colors = random.sample(colors, n_cls_preds)
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:    # data in detection array!
            print ('\t+ Label: %s, Conf: %.5f' % (classes[int(cls_pred)], cls_conf.item()))

            # Rescale coordinates to original dimensions
            box_h = ((y2 - y1) / unpad_h) * img.shape[0]
            box_w = ((x2 - x1) / unpad_w) * img.shape[1]
            y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
            x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]

            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            # Create a Rectangle patch
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor='none')
            # Add the bbox to the plot
            ax.add_patch(bbox)
            # Add label
            plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top', bbox={'color': color, 'pad': 0})

    # Save generated image with detections
    plt.axis('off')
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
   
    plt.savefig(output_folder + "/%d.png" % (img_i), dpi=1000, bbox_inches='tight', pad_inches=0.0)
    
    # Instead of saving the file, convert to numpy array
    fig.canvas.draw()
    pic_numpy = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    pic_numpy = pic_numpy.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.close()

#### Parse through the video frame by frame

In [ ]:
import cv2
import os

In [ ]:
video_file = "input-video.avi"
output_folder = "OUTPUT_VIDEO_FRAMES"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Create video capture object to read video frames
vid_read = cv2.VideoCapture(video_file)
# Create video writer object
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
vid_write = cv2.VideoWriter('OUTPUT_VIDEO.avi', fourcc, 30.0, (1280, 720))

In [ ]:
success, frame = vid_read.read()
print("Data type of frame: ", type(frame))
print("Frame size: ", frame.shape, "\tin width height channels")
frame_number = 0
while success:
    # Save individual frames
    #cv2.imwrite(output_folder + "/frame%d.jpg" % frame_number, image)     # save frame as JPEG file     
    
    # Process one frame ...
    frame = cv2.flip(frame,0)
    # write the flipped frame
    vid_write.write(frame)
    
     
    
    success, frame = vid_read.read()
    #frame_number += 1
    
vid_read.release()
vid_write.release()
print("Done!")